In [ ]:
!uv add pydantic-ai

In [2]:
from gitsource import GithubRepositoryDataReader
from minsearch import AppendableIndex

reader = GithubRepositoryDataReader(
    repo_owner="evidentlyai",
    repo_name="docs",
    allowed_extensions={"md", "mdx"},
)
files = reader.read()

parsed_docs = [doc.parse() for doc in files]

index = AppendableIndex(
    text_fields=["title", "description", "content"],
    keyword_fields=["filename"]
)
index.fit(parsed_docs)

In [5]:
from minsearch import Highlighter, Tokenizer
from minsearch.tokenizer import DEFAULT_ENGLISH_STOP_WORDS

In [6]:
stopwords = DEFAULT_ENGLISH_STOP_WORDS | {'evidently'}

highlighter = Highlighter(
    highlight_fields=['content'],
    max_matches=3,
    snippet_size=50,
    tokenizer=Tokenizer(stemmer='snowball', stop_words=stopwords)
)

In [7]:
file_index = {doc['filename']: doc['content'] for doc in parsed_docs}

In [8]:
from typing import Any, Dict, List


class SearchTools:
    """
    Provides search and file retrieval utilities over an indexed data store.
    """

    def __init__(
        self,
        index: Any,
        highlighter: Any,
        file_index: Dict[str, str]
    ) -> None:
        """
        Initialize the SearchTools instance.

        Args:
            index: Searchable index providing a `search` method.
            highlighter: Highlighter used to annotate search results.
            file_index (Dict[str, str]): Mapping of filenames to file contents.
        """
        self.index = index
        self.highlighter = highlighter
        self.file_index = file_index

    def search(self, query: str) -> List[Dict[str, Any]]:
        """
        Search the index for results matching a query and highlight them.

        Args:
            query (str): The search query to look up in the index.

        Returns:
            List[Dict[str, Any]]: A list of highlighted search result objects.
        """
        search_results = self.index.search(
            query=query,
            num_results=5
        )

        return self.highlighter.highlight(query, search_results)

    def get_file(self, filename: str) -> str:
        """
        Retrieve a file's contents by filename.

        Args:
            filename (str): The filename of the file to retrieve.

        Returns:
            str: The file contents if found, otherwise an error message.
        """
        if filename in self.file_index:
            return self.file_index[filename]
        return f"file {filename} does not exist"


In [9]:
search_tools = SearchTools(index, highlighter, file_index)

In [10]:
instructions = """
You're a documentation assistant.

Answer the user question using only the documentation knowledge base.

Make 3 iterations:

1) First iteration:
   - Perform one search using the search tool to identify potentially relevant documents.
   - Explain (in 2–3 sentences) why this search query is appropriate for the user question.

2) Second iteration:
   - Analyze the results from the previous search.
   - Based on the filenames or documents returned, perform:
       - Up to 2 additional search queries to refine or expand coverage, and
       - One or more get_file calls to retrieve the full content of the most relevant documents.
   - For each search or get_file call, explain (in 2–3 sentences) why this action is necessary and how it helps answer the question.

3) Third iteration:
   - Analyze the retrieved document contents from get_file.
   - Synthesize the information from these documents into a final answer to the user.

IMPORTANT:
- At every step, explicitly explain your reasoning for each search query or file retrieval.
- Use only facts found in the documentation knowledge base.
- Do not introduce outside knowledge or assumptions.
- If the answer cannot be found in the retrieved documents, clearly inform the user.

Additional notes:
- The knowledge base is entirely about Evidently, so you do not need to include the word "evidently" in search queries.
- Prefer retrieving and analyzing full documents (via get_file) before producing the final answer.
"""

In [11]:
from pydantic_ai import Agent
from toyaikit.tools import get_instance_methods

In [15]:
# tools = [search_tools.search, search_tools.get_file]
tools = get_instance_methods(search_tools)

In [16]:

search_agent = Agent(
    name='search',
    model='openai:gpt-4o-mini',
    instructions=instructions,
    tools=tools
)
    

In [3]:
query = 'how do I use evidently to monitor my machine learning models?'

In [19]:
result = await search_agent.run(query)

In [22]:
print(result.output)

3) From the retrieved documents, I found the following relevant information about how to use Evidently to monitor your machine learning models.

Evidently provides several options for monitoring machine learning models effectively. The platform supports batch monitoring jobs, which is suitable for batch ML pipelines, regression testing, and near real-time ML systems. You can create an evaluation pipeline using tools such as Python scripts or orchestration tools like Airflow. This involves scheduling evaluations, running metric calculations, and visualizing the results on a dashboard. 

Additionally, Evidently also offers tracing capabilities for applications powered by large language models (LLMs). This method involves instrumenting your application to capture relevant data and scheduling evaluations that can run automatically. This makes data management simpler and allows for easier re-evaluation of model performance.

**Key Steps to Monitor Your Machine Learning Models Using Evidentl

In [25]:
messages = result.all_messages()

In [31]:
result.usage()

RunUsage(input_tokens=4031, output_tokens=571, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, requests=3, tool_calls=3)

In [37]:
for m in messages:
    print(m.kind)
    for p in m.parts:
        part_kind = p.part_kind
        if part_kind == 'user-prompt':
            print('USER:', p.content)
        if part_kind == 'tool-call':
            print('TOOL CALL:', p.tool_name, p.args)
        if part_kind == 'tool-return':
            print('TOOL RETURN:', p.tool_name)
        if part_kind == 'text':
            print(p.content)
    print()

request
USER: how do I use evidently to monitor my machine learning models?

response
TOOL CALL: search {"query":"monitor machine learning models"}

request
TOOL RETURN: search

response
1) I searched for "monitor machine learning models" because it directly relates to your inquiry about using Evidently for monitoring machine learning models. This search should yield relevant documents about monitoring capabilities and practices within the tools or frameworks provided by Evidently.

2) Analyzing the search results, the document titled "Overview" discusses how production AI quality monitoring works, which likely includes procedures and examples relevant to monitoring models. I'll perform a get_file call on this document to retrieve its contents.

Additionally, the document titled "What is Evidently?" may contain an overview of monitoring and evaluation tools within the platform. I'll perform another get_file call to access this document.

Here’s the request to retrieve the contents of t

In [38]:
new_prompt = 'show me the code'

In [39]:
result2 = await search_agent.run(new_prompt, message_history=messages)

In [41]:
print(result2.output)

Based on the retrieved documents, here are code examples to help you get started with using Evidently for monitoring machine learning models.

### 1. Quickstart for Monitoring ML Models

To monitor your machine learning models, you can follow a basic setup where you run evaluations and visualize results. Below is a sample code snippet that illustrates how to use Evidently for monitoring data quality and detecting data drift:

```python
from evidently import ColumnMapping
from evidently.report import Report
from evidently.runners import Runner

# Load your data and reference
reference_data = ...
current_data = ...

# Define column mapping
column_mapping = ColumnMapping(
    target="your_target_column_name",
    prediction="your_predictions_column_name",
    numerical_features=["numerical_feature_1", "numerical_feature_2"],
    categorical_features=["categorical_feature_1"]
)

# Create a report
report = Report()

# Add metrics to the report
report.add_metric("DataDrift", current_data, re

In [46]:
def print_messages(messages):
    for m in messages:
        print(m.kind)
        for p in m.parts:
            part_kind = p.part_kind
            if part_kind == 'user-prompt':
                print('USER:', p.content)
            if part_kind == 'tool-call':
                print('TOOL CALL:', p.tool_name, p.args)
            if part_kind == 'tool-return':
                print('TOOL RETURN:', p.tool_name)
            if part_kind == 'text':
                print(p.content)
        print()

In [47]:
print_messages(result2.new_messages())

request
USER: show me the code

response
TOOL CALL: search {"query":"code examples"}

request
TOOL RETURN: search

response
1) I searched for "code examples" to find specific coding implementations related to Evidently. This will help me locate documents that provide sample code or examples demonstrating how to use the library for monitoring machine learning models effectively.

2) The search results indicate that the document titled "Tutorials and guides" contains end-to-end examples of specific workflows and will likely include relevant code snippets. Therefore, I will perform a get_file call for this document to retrieve its content.

Additionally, the document titled "All Metrics" mentions reference code examples associated with dataset-level evaluations. I'll retrieve this document as well.

Here’s the request to retrieve the contents of these two documents:

- Get file for "examples/introduction.mdx"
- Get file for "metrics/all_metrics.mdx"

These documents should provide practic

In [58]:
from pydantic_ai import RunUsage

In [59]:
messages = []
usage = RunUsage()

while True:
    user_prompt = input('You:')
    if user_prompt.lower().strip() == 'stop':
        break

    result = await search_agent.run(user_prompt, message_history=messages)
    usage = usage + result.usage()

    print_messages(result.new_messages())
    messages.extend(result.new_messages())

You: dsasboard evidently


request
USER: dsasboard evidently

response
1) I will search for "dashboard" as it is a likely relevant term related to managing or visualizing data within the context of the platform. This search will help uncover documentation related to dashboard features or functionalities, which should inform the user regarding their inquiry.

```json
{"query": "dashboard"}
```

2) Now, I'll analyze the results from the previous search to determine their relevance. Based on the items returned, I will perform an additional search query for "dashboard features" to refine the results specifically around the functionalities available. Additionally, I will retrieve content from the document that seems most relevant to the dashboard topic to better address any specific information the user might be seeking.

```json
{"query": "dashboard features"}
```

3) Once the relevant documents are retrieved, I will read through their content to provide a comprehensive answer, synthesizing the information effective

You: do the actual search


request
USER: do the actual search

response
TOOL CALL: search {"query":"dashboard"}

request
TOOL RETURN: search

response
The search for "dashboard" returned several relevant documents that provide insights into the features and functionalities related to dashboards. The documents seem to cover different aspects such as adding panels, an overview of dashboards, and legacy panel types. This breadth of results indicates that there is useful information that can be synthesized to answer the user’s question effectively.

Next, I will perform an additional search for "dashboard features" to further hone in on specific functionalities, which may provide detailed explanations or guidelines relevant to dashboard capabilities. Additionally, I will retrieve content from the most relevant document titled "Overview," which is crucial for understanding what a dashboard is and its core features.

Let's execute these next actions.
TOOL CALL: search {"query": "dashboard features"}
TOOL CALL: get_fil

You: stop


In [60]:
usage

RunUsage(input_tokens=4804, cache_read_tokens=1280, output_tokens=811, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, requests=4, tool_calls=3)

In [70]:
from pydantic_ai.messages import FunctionToolCallEvent

class NamedCallback:

    def __init__(self, agent):
        self.agent_name = agent.name

    async def print_function_calls(self, ctx, event):
        # Detect nested streams
        if hasattr(event, "__aiter__"):
            async for sub in event:
                await self.print_function_calls(ctx, sub)
            return

        if isinstance(event, FunctionToolCallEvent):
            tool_name = event.part.tool_name
            args = event.part.args
            print(f"TOOL CALL ({self.agent_name}): {tool_name}({args})")

    async def __call__(self, ctx, event):
        return await self.print_function_calls(ctx, event)

In [71]:
callback = NamedCallback(search_agent) 

In [72]:
result = await search_agent.run(
    "how do I build dashboads with evidently?",
    event_stream_handler=callback
)

TOOL CALL (search): search({"query":"build dashboards"})
TOOL CALL (search): search({"query": "dashboard management"})
TOOL CALL (search): search({"query": "add dashboard panels"})
TOOL CALL (search): get_file({"filename": "docs/platform/dashboard_add_panels.mdx"})
TOOL CALL (search): get_file({"filename": "docs/platform/dashboard_add_panels_ui.mdx"})
TOOL CALL (search): get_file({"filename": "docs/platform/dashboard_panel_types.mdx"})
